In [ ]:
'''
this notebook is defining how the cxc motifs are found
it also determines if the motif is past inside or before the IPR region 
This generated a annotated data set which I later added information about MauA into after the fact in notebook 2_4

This did not change this annotation and there was no filtering at that step so it was tacked on to the first dataset and moved
forward

'''

In [ ]:
import pandas as pd
import re
from Bio.SeqUtils.ProtParam import ProteinAnalysis


In [ ]:
import Bio
from Bio import SeqIO
from io import StringIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import sys, errno, re, json, ssl
from urllib import request
from urllib.error import HTTPError
from time import sleep
import os
import re
import csv
import pandas as pd
from Bio import Entrez

In [ ]:
# bring in the full raw data set
FBDS = pd.read_excel(r'C:\PATH\1_full_set_raw_10_2024.xlsx')

In [ ]:
'''
below I am using the same function as before so that I can pull the IPR deliminating regions in the sequence
and then search for a CXC in that region. This was only done for entries with MauE. Everything else will show as NaN in
the dataframe.
'''

def fetch_data(BASE_URL):

    context = ssl._create_unverified_context()
    next_url = BASE_URL
    returned_df = pd.DataFrame()  
    
    while next_url:
        try:
            req = request.Request(next_url, headers={"Accept": "application/json"})
            res = request.urlopen(req, context=context)

            if res.status == 408:
                sleep(61)
                continue
            elif res.status == 204:
                break

            payload = json.loads(res.read().decode())
            next_url = payload.get("next", None)
                
            for item in payload["results"]:
                IPR = 'IPR'
                test_string =  item['metadata']['accession']
                accession = item['proteins'][0]['accession']
                returned_df.at[0,'Accession'] = accession
                if IPR in test_string:
                    #print('accession is IPR entry')
                    #print('num fragments: ', len(item['proteins'][0]['entry_protein_locations'][0]['fragments']))
                    for i in item['proteins'][0]['entry_protein_locations'][0]['fragments']:
                        region_string = str(i['start']) + '_'+str(i['end'])
                        #print(region_string)
                        col_name = test_string + '_{}'.format(item['proteins'][0]['entry_protein_locations'][0]['fragments'].index(i))
                        #print(col_name)
                        returned_df.at[0,col_name] = region_string
                
            
                    
                else:
                    continue
            #display(returned_df)
         
                
            

            sleep(1)  

        except HTTPError as e:
            if e.code == 408:
                sleep(61)
                continue
            else:
                print(f"Error occurred: {e}")
                break

    return returned_df


In [ ]:
#this section pulls for maue only
base = "https://www.ebi.ac.uk/interpro/api/entry/all/protein/"
db1 = 'unreviewed/'
db2 = 'reviewed/'
db3 = 'UniProt/'
mask = FBDS['IPR009908'] == 'IPR009908'
FBDS2 = FBDS.loc[mask]
accession_list = list(FBDS2['Accession'])
print(len(accession_list))


In [ ]:
#small check
print(accession_list[1])

In [ ]:
#This section creates a dataframe using fetch_data that has every IPR associated with each MAUE entry in the 
#full bacterial dataset FBDS
#this includes the IPR start and end which will be used to get the active site motif metadata

IPR_info_FBDS = pd.DataFrame()
for i in accession_list:
    try:
        url= base+db1+i
        #print(url)
        new_row = fetch_data(url)
        IPR_info_FBDS = pd.concat([IPR_info_FBDS,new_row])
    except:
        try:
            url= base+db2+i
            new_row = fetch_data(url)
            IPR_info_FBDS = pd.concat([IPR_info_FBDS,new_row])
        except:
            try:
                
                url= base+db3+i
                new_row = fetch_data(url)
                IPR_info_FBDS = pd.concat([IPR_info_FBDS,new_row])
            except:
                print('not found')
                
IPR_df2 = IPR_info_FBDS.reset_index(drop=True)


In [ ]:
#small check
from IPython.display import display
display(IPR_df2)

In [ ]:
#save metadata
IPR_df2.to_excel(r'C:\PATH\2b_IPR_metadata_for_all_maue.xlsx')

In [ ]:
'''
Now that we have the full raw data set and the metadata associated with the IPR
we can search for the active site
'''

In [ ]:
#this sets the accession as the index of the fbds then searches the accession from IPR_df2 

FBDS3 = FBDS.set_index('Accession')
display(FBDS3)

In [ ]:
#add the columns

for col in ['CXC_at_all', 'CXC_past','CXC_before','CXC_inside_IPR']:
    if col not in FBDS.columns:
        FBDS3[col] = 0

In [ ]:
## I used this to add metadata on the location of the cxc in the sequence and the center amino acid
## you need to filter cxc in the motif but there is no X amino acid data because the logo plot covers that
#searching by IPR_df2 because that contains all the entries for maue that I could pull the metadata on and 
#leaves out the entries that cant be found but were there in the fasta pull (I checked those and those entries are empty for metadata)



for ind in IPR_df2.index:
    accession = IPR_df2.at[ind, 'Accession']
    acc_upper = accession.upper()
    print(accession, acc_upper)
    seq = FBDS3.at[acc_upper, 'Sequence']
    print(seq)
    location_string = IPR_df2.at[ind,'IPR009908_0']
    try:
        
        start = int(location_string.split('_')[0])
        end = int(location_string.split('_')[1])

   

        # Detect all CXC motifs
        matches = re.finditer(r'C[A-Z]C', seq)
        for match in matches:
            loc = match.start()
            motif = match.group()
        

            if (loc >= start) and (loc <= end):
                FBDS3.at[acc_upper,'CXC_at_all'] += 1
                FBDS3.at[acc_upper,'CXC_inside_IPR'] += 1
            
            elif loc > end:
                FBDS3.at[acc_upper,'CXC_at_all'] += 1
                FBDS3.at[acc_upper,'CXC_past'] += 1
           
            elif loc < start:
                FBDS3.at[acc_upper,'CXC_at_all'] += 1
                FBDS3.at[acc_upper,'CXC_before'] += 1
    except:
        continue

           
            

In [ ]:
#check dataframe
display(FBDS3)

In [ ]:
#export the FBDS with the CXC metadata
#now it is ready to filter by species and phylum 
#this is where the original CXC annotation happened, and MauA was added in the second notebook after the fact since 
#it did not change any annotations or filtering at that step
FBDS3.to_excel(r'C:\PATH\2_FBDS_annotated_CXC.xlsx')

In [ ]:
'''
its worth noting that the regex term will fill the cxc_at_all tab with the number of cxc hits it has. depending on how you want
to filter down you can then process the rest of the data set as we have. but It is our opinion that any MauE with a cxc at all 
within the IPR region is an active candidate for the data analysis. if the cxc motif is outside the IPR region or if there is 
no cxc motif it is regarded as a false positive pull from the interpro database and is not used in the analysis.
'''